In [2]:
import numpy as np
from scipy.optimize import linprog

def find_mix_strategy(payoff1, payoff2):
    payoff11 = [i + [0]*len(payoff2) +[-1,0] for i in payoff1]
    payoff21 = [[0]*len(payoff1) +i + [0,-1] for i in payoff2]
    t=payoff11+payoff21
    A_ub = np.array(t)
    b_ub = np.zeros(len(t))
    
    e = [[1]*len(payoff1) + [0]*len(payoff2) +[0,0]] + [[0]*len(payoff1) +[1]*len(payoff2) + [0,0]]
    A_eq = np.array(e)
    b_eq = np.ones(2)
    c = np.array([0] * len(t) + [1,1])
    res = linprog(c, A_ub=A_ub, b_ub=b_ub,bounds=(0, None), A_eq=A_eq,b_eq=b_eq)
    return f"Chiến lược của người 1:{res.x[len(payoff1):len(payoff1)+len(payoff2)]}\nChiến lược của người 2:{res.x[:len(payoff1)]},\nKỳ vọng của người 1: {res.x[len(payoff1)+len(payoff2):len(payoff1)+len(payoff2)+1]},\nKỳ vọng của người 2: {res.x[len(payoff1)+len(payoff2)+1:len(payoff1)+len(payoff2)+2]}"

payoff1 = [[1,-2,-3],[-1,2,-3],[-1,-2,3]]
payoff2 = [[1,-1,-1],[-2,2,-2],[-3,-3,3]]
print(find_mix_strategy(payoff1,payoff2))

Chiến lược của người 1:[0.5 0.  0.5]
Chiến lược của người 2:[0.  0.6 0.4],
Kỳ vọng của người 1: [0.],
Kỳ vọng của người 2: [0.]


In [58]:
from numpy.random import random
from random import randint, choices
#epsilon greedy strategy the agent visit new states with epsilon probability
EPSILON = 0.1
#number of bandits
BANDITS = 10
#number of iterations.
EPISODES = 100000

payoff = [[-9,-1],[0,-6]]

In [56]:
class Bandit():
    def __init__(self, probability, n2):
        # Qk(a) stores the mean of rewards
        self.q = 0
        #k means how many times action a (so the bandit) was chosen in the past
        self.k = 0
        # probability distribution
        self.probability = probability
        self.n2 = n2

    def get_reward(self):
        nam = choices([0,1], weights=[self.n2,1-self.n2], k=1)[0]
        if random() < self.probability:
            return payoff[0][nam]
        else:
            return payoff[1][nam]

class NArmedBandit():
    def __init__(self,n2):
        self.best = []
        self.bandits = []
        self.n2 = n2
        self.l = [i/10 for i in range(10)]
        for i in self.l:
            self.bandits.append(Bandit(i,self.n2))

    def run(self):
        for i in range(EPISODES):
            bandit = self.bandits[self.select_bandit()]
            reward = bandit.get_reward()
            self.update(bandit, reward)
            # print(f'Iteration {i}, bandit {bandit.probability} with Q value {bandit.q}')        

    def select_bandit(self):
        # this is the epsilon-greedy strategy
        # with epsilon probability the agent explore otherwise it exploits
        if random() < EPSILON:
            bandit_index = randint(0, BANDITS - 1)
        else:
            bandit_index = self.get_bandit_max_q()
        return bandit_index
    
    def update (self, bandit, reward):
        bandit.k = bandit.k + 1
        bandit.q = bandit.q + (1 / (1 + bandit.k))* (reward - bandit.q)    

    def get_bandit_max_q(self):
        # we find the bandit with max Q(a) value for the greedy exploitation
        # we need the index of the bandit with max Q(a)
        max_q_bandit_index = 0
        max_q = self.bandits[max_q_bandit_index].q
        for i in range (1, BANDITS):
            if self.bandits[i].q> max_q:
                max_q = self.bandits[i].q
                max_q_bandit_index = i
        return max_q_bandit_index        

    #show statistics: how many times the given bandit was chosen
    def show_statistics(self):
        for i in range (BANDITS):
            self.best.append(self.bandits[i].k)
            print(f'Strategy {i} with k: {self.bandits[i].k}')   
    
    def best_stra(self):
        for i in range (BANDITS):
            self.best.append(self.bandits[i].k)
        print(f'Best prob is {self.l[self.best.index(max(self.best))]}')
        

In [60]:
a=NArmedBandit(1)
a.run()
a.show_statistics()
a.best_stra()

Strategy 0 with k: 91062
Strategy 1 with k: 946
Strategy 2 with k: 981
Strategy 3 with k: 988
Strategy 4 with k: 999
Strategy 5 with k: 1017
Strategy 6 with k: 1010
Strategy 7 with k: 984
Strategy 8 with k: 1033
Strategy 9 with k: 980
Best prob is 0.0
